# Configuration

Define settings for remote services, create `appsettings.Development.json` using `appsettings.json` as template.

You need to manually give yourself `Cognitive Services Contributor` rights on the `Azure OpenAI` resource if you not already have it.

Also ensure to execute `azd auth login` to have access to the Azure resources.

> This example is based on [How-To: Coordinate Agent Collaboration using Agent Group Chat](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/examples/example-agent-collaboration?pivots=programming-language-csharp).


# Install and Import Required Packages
Install and import the necessary packages using NuGet.

In [7]:
// Install the necessary packages using NuGet
#r "nuget: Azure.Identity, 1.13.1"
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.33.0"

// Import the necessary libraries
using System;
using System.Net;
using System.ComponentModel;
using System.Diagnostics;
using System.IO;
using System.Text.Json;
using System.Threading.Tasks;
using Azure.Identity;
using Microsoft.DotNet.Interactive;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.History;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;

Installed Packages Azure.Identity, 1.13.1 Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Binder, 9.0.0 Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.0 Microsoft.Extensions.Configuration.UserSecrets, 9.0.0 Microsoft.SemanticKernel, 1.33.0 Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.33.0

# Create Kernel Builder
Create a Kernel builder instance.

In [8]:
var configBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("./appsettings.json", optional: false)
    .AddJsonFile("./appsettings.Development.json", optional: false);
    
var configuration = configBuilder.Build();

IKernelBuilder builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(
	configuration["ChatModelDeployment"],
    configuration["Endpoint"],
	new AzureDeveloperCliCredential());

Microsoft.SemanticKernel.Kernel kernel = builder.Build();

public class Tools
{
    [KernelFunction]
    [Description("Copies the provided content to the console output.")]
    public static void WriteConsoleOutput(string content)
    {
        if (string.IsNullOrWhiteSpace(content))
        {
            return;
        }

        Console.Write(content);
    }
}

Microsoft.SemanticKernel.Kernel toolKernel = kernel.Clone();
toolKernel.Plugins.AddFromType<Tools>()

8.0.11

[ Tools.WriteConsoleOutput ] FunctionCount 1 Name Tools Description (values) index value 0 Tools.WriteConsoleOutput Name WriteConsoleOutput PluginName Tools Description Copies the provided content to the console output. Metadata Microsoft.SemanticKernel.KernelFunctionMetadata Name WriteConsoleOutput PluginName Tools Description Copies the provided content to the console output. Parameters index value 0 Microsoft.SemanticKernel.KernelParameterMetadata Name content Description DefaultValue <null> IsRequired True ParameterType System.String Schema {"type":"string"} RootElement {
 "type": "string"
} ReturnParameter Microsoft.SemanticKernel.KernelReturnParameterMetadata Description ParameterType System.Void Schema <null> AdditionalProperties (empty) ExecutionSettings <null>


# Agent Definitions

In [9]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

const string ReviewerName = "Reviewer";
const string WriterName = "Writer";

ChatCompletionAgent agentReviewer =
    new()
    {
        Name = ReviewerName,
        Instructions =
            """
            Your responsiblity is to review and identify how to improve user provided content.
            If the user has providing input or direction for content already provided, specify how to address this input.
            Never directly perform the correction or provide example.
            Once the content has been updated in a subsequent response, you will review the content again until satisfactory.
            Always copy satisfactory content to the console output using available tools and inform user.

            RULES:
            - Only identify suggestions that are specific and actionable.
            - Verify previous suggestions have been addressed.
            - Never repeat previous suggestions.
            """,
        Kernel = toolKernel,
        Arguments =
            new KernelArguments(
                new AzureOpenAIPromptExecutionSettings() 
                { 
                    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() 
                })
    };

ChatCompletionAgent agentWriter =
    new()
    {
        Name = WriterName,
        Instructions =
            """
            Your sole responsiblity is to rewrite content according to review suggestions.

            - Always apply all review direction.
            - Always revise the content in its entirety without explanation.
            - Never address the user.
            """,
        Kernel = kernel,
    };

# Chat Definitions

In [10]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

KernelFunction selectionFunction =
    AgentGroupChat.CreatePromptFunctionForStrategy(
        $$$"""
        Examine the provided RESPONSE and choose the next participant.
        State only the name of the chosen participant without explanation.
        Never choose the participant named in the RESPONSE.

        Choose only from these participants:
        - {{{ReviewerName}}}
        - {{{WriterName}}}

        Always follow these rules when choosing the next participant:
        - If RESPONSE is user input, it is {{{ReviewerName}}}'s turn.
        - If RESPONSE is by {{{ReviewerName}}}, it is {{{WriterName}}}'s turn.
        - If RESPONSE is by {{{WriterName}}}, it is {{{ReviewerName}}}'s turn.

        RESPONSE:
        {{$lastmessage}}
        """,
        safeParameterNames: "lastmessage");


const string TerminationToken = "yes";

KernelFunction terminationFunction =
    AgentGroupChat.CreatePromptFunctionForStrategy(
        $$$"""
        Examine the RESPONSE and determine whether the content has been deemed satisfactory.
        If content is satisfactory, respond with a single word without explanation: {{{TerminationToken}}}.
        If specific suggestions are being provided, it is not satisfactory.
        If no correction is suggested, it is satisfactory.

        RESPONSE:
        {{$lastmessage}}
        """,
        safeParameterNames: "lastmessage");        

# Creating Group Chat

In [11]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

ChatHistoryTruncationReducer historyReducer = new(1);

AgentGroupChat chat =
    new(agentReviewer, agentWriter)
    {
        ExecutionSettings = new AgentGroupChatSettings
        {
            SelectionStrategy =
                new KernelFunctionSelectionStrategy(selectionFunction, kernel)
                {
                    // Always start with the editor agent.
                    InitialAgent = agentReviewer,
                    // Save tokens by only including the final response
                    HistoryReducer = historyReducer,
                    // The prompt variable name for the history argument.
                    HistoryVariableName = "lastmessage",
                    // Returns the entire result value as a string.
                    ResultParser = (result) => result.GetValue<string>() ?? agentReviewer.Name
                },
            TerminationStrategy =
                new KernelFunctionTerminationStrategy(terminationFunction, kernel)
                {
                    // Only evaluate for editor's response
                    Agents = [agentReviewer],
                    // Save tokens by only including the final response
                    HistoryReducer = historyReducer,
                    // The prompt variable name for the history argument.
                    HistoryVariableName = "lastmessage",
                    // Limit total number of turns
                    MaximumIterations = 12,
                    // Customer result parser to determine if the response is "yes"
                    ResultParser = (result) => result.GetValue<string>()?.Contains(TerminationToken, StringComparison.OrdinalIgnoreCase) ?? false
                }
        }
    };

Console.WriteLine("Ready!");

Ready!


# Chat Loop

You can test it with the following

> Elephants are the largest land mamma4ls on earth and have distinctly massive bodies, large ears, and long trunks. They use their trunks to pick up objects, trumpet warnings, greet other elephants, or suck up water for drinking or bathing, among other uses. Both male and female African elephants grow tusks and each individual can either be left- or right-tusked, an5d the one they use more is usually smaller because of wear and tear. Elephant tusks serve many purposes. These extended teeth can be used to protect the elephant's ztrunk, lift and move objects, gather food, and strip bark from trees. They can also be used for defense. During times of droughdd, elephants even use their tusks to dig holes to find water underground.

In [12]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

bool isComplete = false;
do
{
    Console.WriteLine();
    Console.Write("> ");
    var input = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Chat:");
    if (string.IsNullOrWhiteSpace(input))
    {
        continue;
    }
    input = input.Trim();
    if (input.Equals("EXIT", StringComparison.OrdinalIgnoreCase))
    {
        isComplete = true;
        break;
    }

    if (input.Equals("RESET", StringComparison.OrdinalIgnoreCase))
    {
        await chat.ResetAsync();
        Console.WriteLine("[Converation has been reset]");
        continue;
    }

    if (input.StartsWith("@", StringComparison.Ordinal) && input.Length > 1)
    {
        string filePath = input.Substring(1);
        try
        {
            if (!File.Exists(filePath))
            {
                Console.WriteLine($"Unable to access file: {filePath}");
                continue;
            }
            input = File.ReadAllText(filePath);
        }
        catch (Exception)
        {
            Console.WriteLine($"Unable to access file: {filePath}");
            continue;
        }
    }

    Console.WriteLine();
    Console.WriteLine($"User: {input}");
    chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));

    chat.IsComplete = false;

    try
    {
        await foreach (ChatMessageContent response in chat.InvokeAsync())
        {
            Console.WriteLine();
            Console.WriteLine($"{response.AuthorName.ToUpperInvariant()}:{Environment.NewLine}{response.Content}");
        }
    }
    catch (HttpOperationException exception)
    {
        Console.WriteLine(exception.Message);
        if (exception.InnerException != null)
        {
            Console.WriteLine(exception.InnerException.Message);
            if (exception.InnerException.Data.Count > 0)
            {
                Console.WriteLine(JsonSerializer.Serialize(exception.InnerException.Data, new JsonSerializerOptions() { WriteIndented = true }));
            }
        }
    }
} while (!isComplete);


> 
User: Elephants are the largest land mamma4ls on earth and have distinctly massive bodies, large ears, and long trunks. They use their trunks to pick up objects, trumpet warnings, greet other elephants, or suck up water for drinking or bathing, among other uses. Both male and female African elephants grow tusks and each individual can either be left- or right-tusked, an5d the one they use more is usually smaller because of wear and tear. Elephant tusks serve many purposes. These extended teeth can be used to protect the elephant's ztrunk, lift and move objects, gather food, and strip bark from trees. They can also be used for defense. During times of droughdd, elephants even use their tusks to dig holes to find water underground.

REVIEWER:
Here are some suggestions to improve the content:

1. Correct the typographical errors in words like "mamma4ls", "an5d", "ztrunk", and "droughdd".
2. Split the long sentence for better readability: "Both male and female African elephants grow tu

Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 92
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 45
   at Submission#12.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)